In [2]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset, MexicoDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np 
import os
from aif360.algorithms.inprocessing.active_fairness import ActiveFairness
from aif360.algorithms.inprocessing.active_fairness import calibrate_probs
from sklearn.ensemble import RandomForestClassifier
from copy import deepcopy

Import error: cannot import name 'Model'


In [3]:
# Get the dataset and split into train and test

privileged_groups = [{'young': 1}]
unprivileged_groups = [{'young': 0}]


dataset_orig = MexicoDataset()



In [4]:
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.8], shuffle=True)


In [5]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Testing Dataset shape"))
print(dataset_orig_test.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(56244, 186)


#### Testing Dataset shape

(14061, 186)


#### Favorable and unfavorable labels

...

['ftype', 'urban', 'young']


#### Privileged and unprivileged protected attribute values

[array([1.]), array([1.]), array([1.])] [array([0.]), array([0.]), array([0.])]


#### Dataset feature names

['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30', 'x31', 'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39', 'x40', 'x41', 'x42', 'x43', 'x44', 'x45', 'x46', 'x47', 'x48', 'x49', 'x50', 'x51', 'x52', 'x53', 'x54', 'x55', 'x56', 'x57', 'x58', 'x59', 'x60', 'x61', 'x62', 'x63', 'x64', 'x65', 'x66', 'x67', 'x68', 'x69', 'x70', 'x71', 'x72', 'x73', 'x74', 'x75', 'x76', 'x77', 'x78', 'x79', 'x80', 'x81', 'x82', 'x83', 'x84', 'x85', 'x86', 'x87', 'x88', 'x89', 'x90', 'x91', 'x92', 'x93', 'x94', 'x95', 'x96', 'x97', 'x98', 'x99', 'x100', 'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x107', 'x108', 'x109', 'x110', 'x111', 'x112', 'x113', 'x114', 'x115', 'x116', 'x117', 'x118', 'x119', 'x120', 'x121', 'x122', 'x123', 'x124', 'x125', 'x126', 'x127', 'x128', 'x129', 'x130', 'x131', 'x132', 'x133', 'x134', 'x135', 'x136', 'x137', 'x138', 'x1

In [6]:
# Metric for the original dataset

metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Original training dataset

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.095692
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.112021


In [7]:
min_max_scaler = MaxAbsScaler()
dataset_orig_train.features = min_max_scaler.fit_transform(dataset_orig_train.features)
dataset_orig_test.features = min_max_scaler.transform(dataset_orig_test.features)
metric_scaled_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Scaled dataset - Verify that the scaling does not affect the group label statistics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_train.mean_difference())
metric_scaled_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_test.mean_difference())


#### Scaled dataset - Verify that the scaling does not affect the group label statistics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.095692
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.112021


### Learn plan classifier without optimization

In [8]:
# Create a random forest model 
n_trees = 16
max_leaf_nodes = 350
'''
We use tranining data to train the RF, and apply Active Fairness for both training dataset (Generate df_p_train,df_qa_train,df_y_train,df_mc_train,df_X_train)
and testing dataset  (Generate df_p_test,df_qa_test,df_y_test,df_mc_test,df_X_test) 
'''

# Create a random forest classifier
rf = RandomForestClassifier(max_leaf_nodes=max_leaf_nodes, n_estimators=n_trees, random_state=0)

AF = ActiveFairness(dataset_orig_train, dataset_orig_test,rf,
                          privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          sensitive_features = ['ftype', 'urban', 'young'],
                          target_label = ['Outcome']
                             )


In [9]:
AF.fit()

In [10]:
dataset_noactivefair_train = AF.predict(True)
dataset_noactivefair_test = AF.predict(False)


In [11]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without active fairness - dataset metrics"))
metric_dataset_noactivefair_train = BinaryLabelDatasetMetric(dataset_noactivefair_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_noactivefair_train.mean_difference())

metric_dataset_noactivefair_test = BinaryLabelDatasetMetric(dataset_noactivefair_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_noactivefair_test.mean_difference())

display(Markdown("#### Plain model - without active fairness - classification metrics"))
classified_metric_noactivefair_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_noactivefair_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_noactivefair_test.accuracy())
TPR = classified_metric_noactivefair_test.true_positive_rate()
TNR = classified_metric_noactivefair_test.true_negative_rate()
bal_acc_noactivefair_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_noactivefair_test)
print("Test set: Disparate impact = %f" % classified_metric_noactivefair_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_noactivefair_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_noactivefair_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_noactivefair_test.theil_index())

#### Plain model - without active fairness - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.166879
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.165569


#### Plain model - without active fairness - classification metrics

Test set: Classification accuracy = 0.780741
Test set: Balanced classification accuracy = 0.740737
Test set: Disparate impact = 0.551996
Test set: Equal opportunity difference = -0.177510
Test set: Average odds difference = -0.128265
Test set: Theil_index = 0.181856


### Use AF algorithm to select features

In [12]:
feat_method = 'feat-imp'
privilige_feature = 'young'
privilige_value = 1
unprivilige_value = 0
privilige_num = 10
unprivilige_num =  10
df_p_test,df_qa_test,df_y_test,df_mc_test,df_X_test = AF.run_algo_in_parallel(
                                                                        feat_method,
                                                                        privilige_feature,
                                                                        privilige_variable_value = privilige_value,
                                                                        unprivilige_variable_value = unprivilige_value,
                                                                        pri_num_feature_fetched = privilige_num,
                                                                        un_pri_num_feature_fetched = unprivilige_num,
                                                                        verbose=0,
                                                                        nr_of_batches=40,
                                                                        batch_size=500,
                                                                        n_jobs=2,
                                                                        save_to_file=False) # Run this code in testing dataset
''' updated above'''


# This is used to run the original code
# df_p_test,df_qa_test,df_y_test,df_mc_test,df_X_test = AF.run_algo_in_parallel(
#                                                                         feat_method,
#                                                                         verbose=0,
#                                                                         nr_of_batches=40,
#                                                                         batch_size=500,
#                                                                         n_jobs=2,
#                                                                         save_to_file=False,
#                                                                         static_threshold = 11) # Run this code in testing dataset

# df_p_train,df_qa_train,df_y_train,df_mc_train,df_X_train = AF.run_algo_in_parallel(
#                                                                         feat_method,
#                                                                         verbose=0,
#                                                                         nr_of_batches=120,
#                                                                         batch_size=500,
#                                                                         n_jobs=4,
#                                                                         save_to_file=False, 
#                                                                         run_on_training=True)  # Run this code in training dataset   


START 0 END 500
START 500 END 1000
START 1000 END 1500
START 1500 END 2000
START 2000 END 2500
START 2500 END 3000
START 3000 END 3500
START 3500 END 4000
START 4000 END 4500
START 4500 END 5000
START 5000 END 5500
START 5500 END 6000
START 6000 END 6500
START 6500 END 7000
START 7000 END 7500
START 7500 END 8000
START 8000 END 8500
START 8500 END 9000
START 9000 END 9500
START 9500 END 10000
START 10000 END 10500
START 10500 END 11000
START 11000 END 11500
START 11500 END 12000
START 12000 END 12500
START 12500 END 13000
START 13000 END 13500
START 13500 END 14000
START 14000 END 14061


' update '

### Static Constraint

In [13]:

threshold = 0.5
dataset_noactive_train = deepcopy(AF.dataset_train)
dataset_noactive_test = deepcopy(AF.dataset_test)
dataset_active_test = deepcopy(AF.dataset_test)




In [14]:
for i in range(len(AF.X_te_sensitiveAtarget)):
    if AF.X_te_sensitiveAtarget.iloc[i, AF.feature2columnmap[privilige_feature]] == privilige_value:
        dataset_active_test.labels[i] = 0 if df_p_test.iloc[i, privilige_num - 1] < threshold else 1
    else:
        dataset_active_test.labels[i] = 0 if df_p_test.iloc[i, unprivilige_num - 1] < threshold else 1
print("Finished")


SyntaxError: invalid syntax (<ipython-input-14-8ee48757e18f>, line 1)

In [ ]:
# Metrics for the dataset from plain model (without noactivefair)
display(Markdown("#### Plain model - without active fairness - dataset metrics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_noactivefair_train.mean_difference())
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_noactivefair_test.mean_difference())

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with active fairness - dataset metrics"))

# metric_dataset_noactivefair_train = BinaryLabelDatasetMetric(dataset_noactive_train, 
#                                              unprivileged_groups=unprivileged_groups,
#                                              privileged_groups=privileged_groups)

# print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_noactivefair_train.mean_difference())

metric_dataset_activefair_test = BinaryLabelDatasetMetric(dataset_active_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_activefair_test.mean_difference())



display(Markdown("#### Plain model - without active fairness - classification metrics"))
print("Test set: Classification accuracy = %f" % classified_metric_noactivefair_test.accuracy())
TPR = classified_metric_noactivefair_test.true_positive_rate()
TNR = classified_metric_noactivefair_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_noactivefair_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_noactivefair_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_noactivefair_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_noactivefair_test.theil_index())



display(Markdown("#### Model - with active fairness - classification metrics"))
classified_metric_active_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_active_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_active_test.accuracy())
TPR = classified_metric_active_test.true_positive_rate()
TNR = classified_metric_active_test.true_negative_rate()
bal_acc_active_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_active_test)
print("Test set: Disparate impact = %f" % classified_metric_active_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_active_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_active_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_active_test.theil_index())